In [1]:
import torch
import torch_geometric as pyg
import collections
from lib.graphs.scores import iterative_mean_spectral_gap as imsg
from typing import *

In [8]:
graph = torch.load("results/density_0.1/cifar10/resnet34/SNIP/2/latest/graph.pth")

In [9]:
print(graph.keys())

dict_keys(['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.downsample.0', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.3.conv1', 'layer2.3.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.downsample.0', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.5.conv1', 'layer3.5.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.2.conv1', 'layer4.2.conv2', 'fc'])


In [16]:
for k, v in graph.items():
    print(k, imsg(v))

conv1 (tensor(0.2794, dtype=torch.float64), tensor(0.3149, dtype=torch.float64))
layer1.0.conv1 (tensor(0.2600, dtype=torch.float64), tensor(0.2552, dtype=torch.float64))
layer1.0.conv2 (tensor(0.2886, dtype=torch.float64), tensor(0.2858, dtype=torch.float64))
layer1.1.conv1 (tensor(0.3010, dtype=torch.float64), tensor(0.2923, dtype=torch.float64))
layer1.1.conv2 (tensor(0.2750, dtype=torch.float64), tensor(0.2721, dtype=torch.float64))
layer1.2.conv1 (tensor(0.2960, dtype=torch.float64), tensor(0.2903, dtype=torch.float64))
layer1.2.conv2 (tensor(0.2990, dtype=torch.float64), tensor(0.2964, dtype=torch.float64))
layer2.0.conv1 (tensor(0.2521, dtype=torch.float64), tensor(0.2660, dtype=torch.float64))
layer2.0.conv2 (tensor(0.2459, dtype=torch.float64), tensor(0.2422, dtype=torch.float64))
layer2.0.downsample.0 (tensor(0.1737, dtype=torch.float64), tensor(0.3369, dtype=torch.float64))
layer2.1.conv1 (tensor(0.2070, dtype=torch.float64), tensor(0.2055, dtype=torch.float64))
layer2.1.con

In [60]:
from typing import List, Optional, Tuple, Union
from torch_geometric.typing import PairTensor
from torch_geometric.utils.mask import index_to_mask
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch import Tensor

def get_subgraph(
    subset: Union[Tensor, List[int]],
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    relabel_nodes: bool = False,
    num_nodes: Optional[int] = None,
    return_edge_mask: bool = False,
) -> Tuple[Tensor, Tensor]:
    r"""Returns the induced subgraph of :obj:`(edge_index, edge_attr)`
    containing the nodes in :obj:`subset`.

    Args:
        subset (LongTensor, BoolTensor or [int]): The nodes to keep.
        edge_index (LongTensor): The edge indices.
        edge_attr (Tensor, optional): Edge weights or multi-dimensional
            edge features. (default: :obj:`None`)
        relabel_nodes (bool, optional): If set to :obj:`True`, the resulting
            :obj:`edge_index` will be relabeled to hold consecutive indices
            starting from zero. (default: :obj:`False`)
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        return_edge_mask (bool, optional): If set to :obj:`True`, will return
            the edge mask to filter out additional edge features.
            (default: :obj:`False`)

    :rtype: (:class:`LongTensor`, :class:`Tensor`)

    Examples:

        >>> edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6],
        ...                            [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5]])
        >>> edge_attr = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
        >>> subset = torch.tensor([3, 4, 5])
        >>> subgraph(subset, edge_index, edge_attr)
        (tensor([[3, 4, 4, 5],
                [4, 3, 5, 4]]),
        tensor([ 7.,  8.,  9., 10.]))

        >>> subgraph(subset, edge_index, edge_attr, return_edge_mask=True)
        (tensor([[3, 4, 4, 5],
                [4, 3, 5, 4]]),
        tensor([ 7.,  8.,  9., 10.]),
        tensor([False, False, False, False, False, False,  True,
                True,  True,  True,  False, False]))
    """

    device = edge_index.device

    if isinstance(subset, (list, tuple)):
        subset = torch.tensor(subset, dtype=torch.long, device=device)
    print(subset.dtype)
    if subset.dtype == torch.bool or subset.dtype == torch.uint8:
        num_nodes = subset.size(0)
    else:
        num_nodes = maybe_num_nodes(edge_index, num_nodes)
        print(num_nodes)
        subset = index_to_mask(subset, size=num_nodes)

    print(subset)
    node_mask = subset
    edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
    edge_index = edge_index[:, edge_mask]
    edge_attr = edge_attr[edge_mask] if edge_attr is not None else None

    if relabel_nodes:
        node_idx = torch.zeros(node_mask.size(0), dtype=torch.long,
                               device=device)
        node_idx[subset] = torch.arange(subset.sum().item(), device=device)
        edge_index = node_idx[edge_index]

    if return_edge_mask:
        return edge_index, edge_attr, edge_mask
    else:
        return edge_index, edge_att

In [69]:
def find_d_left_regular(layer: dict, mindegree: int = 3):
    """return a subgraph of graph which contains at minimum d in/out degree

    :returns: TODO

    """
    graph = layer['graph']
    degree = pyg.utils.degree(graph.edge_index[0], graph.num_nodes)
    d_l = degree[0:layer['dim_in']]
    d_r = degree[layer['dim_in']::]
    degrees = collections.Counter(d_l.tolist())
    for degree, count in degrees.items():
        if count < mindegree or degree == 0.0: continue
        node_mask = d_l == degree
        non_zero_idx = torch.nonzero(node_mask).squeeze()
        right_nodes = torch.arange(d_r.size(0)) + layer['dim_in']
        node_to_keep = torch.cat([non_zero_idx, right_nodes])
        # d-left regular bipartie graph
        subgraph = pyg.utils.subgraph(node_to_keep,
                                      graph.edge_index,
                                      graph.edge_attr,
                                      relabel_nodes=True,
                                      num_nodes = graph.num_nodes
                                     )
        ret = {
            'dim_in':
            count,
            'graph':
            pyg.data.Data(
                edge_index=subgraph[0],
                edge_attr=subgraph[1],
                num_nodes=node_to_keep.size(0),
            ),
        }
        yield ret


In [70]:
for g in find_d_left_regular(graph['layer4.0.conv2']):
    print(g)

{'dim_in': 5, 'graph': Data(edge_index=[2, 2170], edge_attr=[2170, 1], num_nodes=517)}
{'dim_in': 4, 'graph': Data(edge_index=[2, 1584], edge_attr=[1584, 1], num_nodes=516)}
{'dim_in': 3, 'graph': Data(edge_index=[2, 1248], edge_attr=[1248, 1], num_nodes=515)}
{'dim_in': 5, 'graph': Data(edge_index=[2, 1650], edge_attr=[1650, 1], num_nodes=517)}
{'dim_in': 4, 'graph': Data(edge_index=[2, 1376], edge_attr=[1376, 1], num_nodes=516)}
{'dim_in': 4, 'graph': Data(edge_index=[2, 1296], edge_attr=[1296, 1], num_nodes=516)}
{'dim_in': 3, 'graph': Data(edge_index=[2, 1206], edge_attr=[1206, 1], num_nodes=515)}
{'dim_in': 4, 'graph': Data(edge_index=[2, 1032], edge_attr=[1032, 1], num_nodes=516)}
{'dim_in': 5, 'graph': Data(edge_index=[2, 1270], edge_attr=[1270, 1], num_nodes=517)}
{'dim_in': 5, 'graph': Data(edge_index=[2, 1280], edge_attr=[1280, 1], num_nodes=517)}
{'dim_in': 5, 'graph': Data(edge_index=[2, 250], edge_attr=[250, 1], num_nodes=517)}
{'dim_in': 3, 'graph': Data(edge_index=[2, 11

In [12]:
print(degrees)

NameError: name 'degrees' is not defined